In [1]:
# from selenium import webdriver
import re
import requests
import pandas as pd
from bs4 import BeautifulSoup

# OCR for tasting notes (if applicable)
from PIL import Image
import numpy as np
from io import BytesIO
# import easyocr

# reader = easyocr.Reader(['en']) # this needs to run only once to load the model into memory

# Functions

In [6]:
def get_products(url, platform = "Shopify"):
    jsonReq = requests.get(url).json()
    if platform == "Shopify":
        products = jsonReq['products']
    elif platform == "Squarespace":
        products = jsonReq['items']
    return products

In [7]:
# Wix and Woocommerce are special and needs its own method of scraping bc god knows
def build_url(link, platform = "Shopify"):
    if platform == "Shopify":
        productJson = link + "/products.json"
    elif platform == "Squarespace":
        productJson = link + "/?format=json"
    elif platform == "Wix":
        productJson = link
    else:
        productJson = link
    return productJson

In [8]:
def extract_flavor_notes(text, profile):
    # join the list of profiles to search for
    keywords = "|".join(profile)

    # Find all matches in the text
    matches = re.findall(keywords, text, re.IGNORECASE)
    return matches


In [9]:
def get_product_details(storeJsonProducts, platform = "Shopify"):
    all_products = pd.DataFrame()
    if platform == 'Shopify':
        for k,v in storeJsonProducts.items():
            if v.shape[0] != 0:
                title = v.title
                excerpt = v.body_html
                clean_excerpt = [BeautifulSoup(a, "html.parser").get_text() for a in excerpt]
                subdataset = pd.DataFrame(data = {"store": k,
                                                "product": title,
                                                "description": clean_excerpt
                                                })
            else:
                subdataset = pd.DataFrame()
            all_products = pd.concat([all_products, subdataset], axis = 0)
    
    elif platform == "Squarespace":
        for k,v in storeJsonProducts.items():
            if v.shape[0] != 0:
                title = v.title
                excerpt = v.excerpt
                clean_excerpt = [BeautifulSoup(a, "html.parser").get_text() for a in excerpt]
                subdataset = pd.DataFrame(data = {"store": k,
                                                "product": title,
                                                "description": clean_excerpt
                                                })
            else:
                subdataset = pd.DataFrame()
            all_products = pd.concat([all_products, subdataset], axis = 0)
    
    all_products = all_products.reset_index(drop = True)
    return all_products

In [10]:
def parse_image(url):
    response = requests.get(url)
    img = np.array(Image.open(BytesIO(response.content)).convert('L'))
    text = reader.readtext(img)
    text_confident = [t for bbox, t, prob in text if prob > 0.5]
    return text_confident

# Parse Coffee Website List

In [68]:
SGcoffee = pd.read_csv("../sg_coffee_websites.txt", delimiter= "\t", index_col = 0)

# Shopify

In [62]:
shopify_sites = SGcoffee[SGcoffee.Platform == "Shopify"]

In [63]:
shopifyJsonLink = {}
shopifyJsonProducts = {}
for s in shopify_sites.index:
    print(s)
    parseLink = build_url(shopify_sites.loc[s, "Link"])
    shopifyJsonLink[s] = parseLink
    listing = get_products(parseLink)
    shopifyJsonProducts[s] = pd.DataFrame.from_dict(listing)

2 Degrees North
20 grams
Alchemist
Asylum Coffeehouse Filter
Asylum Coffeehouse Espresso
Bettr
Bloom Coffee
Brawn and Brains Espresso
Brawn and Brains Filter
Cata Coffee
Cumulo Coffee
Doubleup Coffee
Dutch Colony Coffee
Flip Coffee Roasters
Fluid Collective
Homeground Coffee Roasters
Kurasu
Luli Roasts
Narrative Coffee Stand
Parchmen & Co
PPP Coffee
Quarter Life Coffee
Rookies Coffee
Round Boy
Shake Coffee
Small Waves
The Community
Tiong Hoe Coffee
Upside Down Coffee
Wasa Coffee
Zerah Coffee Roasters


In [74]:
all_shopify_products = get_product_details(shopifyJsonProducts, platform="Shopify")

In [65]:
nonolist = ['Equipment', 'Brewer', 'Merchandise']

In [82]:
# this works in theory but it's too slow
all_shopify_product_image_description = pd.DataFrame()
for i in shopify_sites[shopify_sites['Scrape_Image']].index[0:1]:
    json_pdt = shopifyJsonProducts[i]
    mask_noscrape = json_pdt[~json_pdt['product_type'].isin(nonolist)]
    imgs_to_scrape = mask_noscrape['images'] # each store's products
    product_name = mask_noscrape['title']

    store_product_texts = pd.DataFrame()
    for prod_name, img_set in zip(product_name, imgs_to_scrape): # for each product
        product_text = []
        for img in img_set: # each image in each product
            url = img['src']
            print(i, prod_name, url)
            img_text = parse_image(url) # convert the image text to string
            product_text += img_text
        clean_product_text = "".join(product_text)
        clean_product_text = clean_product_text.replace("\n", " ")
        product_df = pd.DataFrame(data = {'store': i, 'title': prod_name, 'image_description': clean_product_text}, index = [0])
        store_product_texts = pd.concat([store_product_texts, product_df], axis = 0)
    all_shopify_product_image_description = pd.concat([all_shopify_product_image_description, store_product_texts], axis = 0)

Bettr Eureka https://cdn.shopify.com/s/files/1/0164/9429/7142/files/1_7a8bbcf9-0dcf-4a8e-8f6f-eb6ed4952247.png?v=1701934204
Bettr Eureka https://cdn.shopify.com/s/files/1/0164/9429/7142/files/MadeWithPassionSG_9b52628e-a7d8-4a0a-8bc2-69276c125396.png?v=1707987386


KeyboardInterrupt: 

# Squarespace

In [69]:
sqspace_sites = SGcoffee[SGcoffee.Platform == "Squarespace"]

In [70]:
sqspaceJsonLink = {}
sqspaceJsonProducts = {}
for i in sqspace_sites.index:
    print(i)
    jlnk = build_url(sqspace_sites.loc[i, "Link"], platform="Squarespace")
    sqspaceJsonLink[i] = jlnk
    pdt = get_products(jlnk, platform="Squarespace")
    sqspaceJsonProducts[i] = pd.DataFrame.from_dict(pdt)


Glyph Supply Co
Kyuukei Coffee
Maxi Coffee Bar
Nylon Coffee Roasters
Parallel Coffee Roasters Filter
Parallel Coffee Roasters Espresso
Skill Issue


In [72]:
all_squarespace_products = get_product_details(sqspaceJsonProducts, platform="Squarespace")

In [73]:
all_squarespace_products

,store,product,description
0,Glyph Supply Co,Las Alasitas Bolivia (Espresso),Varietal - Red CaturraProcessing - Washed Most...
1,Glyph Supply Co,Janson 598 Panama,Weight: 100g
2,Glyph Supply Co,Janson 480 Panama,Weight: 100g
3,Glyph Supply Co,Finca Soledad Sidra Ecuador,Weight: 100g
4,Glyph Supply Co,El Vergel Gesha Spirits Colombia,Weight: 100g
...,...,...,...
69,Skill Issue,El Socorro,COFFEE DETAILSRegion: PalenciaProducer: Juan D...
70,Skill Issue,Duwancho,"An Ethiopian with the fines. But washed. Well,..."
71,Skill Issue,Southern Star #5,Impressed by the quality of coffee on the nati...
72,Skill Issue,Capim Branco,"A coffee family spanning 5 generations, Capim ..."


# Writing data to csv

In [152]:
all_data = pd.concat([all_shopify_products, all_squarespace_products], axis=0)
all_data = all_data.reset_index(drop=True)

In [153]:
all_data.to_excel("database_products.xlsx", index=False)

# Extract Flavour Notes

In [169]:
matches_found = pd.DataFrame()
for i in all_data.index:
    text = all_data.loc[i, "description"]
    matches = extract_flavor_notes(text, ['yogurt', 'yoghurt'])
    if len(matches) > 0:
        matches_found = pd.concat([matches_found, all_data.iloc[[i]]], axis = 0)

In [83]:
for k,v in shopifyJsonProducts.items():
    if v.shape[0] != 0:
        title = v.title
        excerpt = v.body_html
        clean_excerpt = [BeautifulSoup(a, "html.parser").get_text() for a in excerpt]
        subdataset = pd.DataFrame(data = {"store": k,
                                        "product": title,
                                        "description": clean_excerpt
                                        })

In [88]:
sqspaceJsonProducts['Glyph Supply Co']

,id,collectionId,recordType,addedOn,updatedOn,displayIndex,starred,passthrough,categoryIds,workflowState,...,pushedServices,pendingPushedServices,recordTypeLabel,systemDataId,systemDataVariants,systemDataSourceType,filename,mediaFocalPoint,colorData,originalSize
0,6723ad6815646034f3d092d2,66e00bac4c639c1118427a9c,11,1728015905740,1730392171694,0,False,False,[66ff6b56fb783433b97287e1],1,...,{},{},store-item,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,66ff76dca4c24b619b59da0b,66e00bac4c639c1118427a9c,11,1728018140658,1729007746705,10,False,False,[66ff6b514c76fd46af3cc5d6],1,...,{},{},store-item,67bb0950-afe8-4342-a589-b63f44d911c5,"638x709,100w,300w,500w",PNG,480598LajoyaAlo-01_2048x.png,"{'x': 0.5, 'y': 0.5, 'source': 3}","{'topLeftAverage': 'ffe8ca', 'topRightAverage'...",638x709
2,66ff75c8a4c24b619b59d996,66e00bac4c639c1118427a9c,11,1728017864783,1729007704182,9,False,False,[66ff6b514c76fd46af3cc5d6],1,...,{},{},store-item,71da3562-f9b2-4217-af46-a19d78722935,"638x709,100w,300w,500w",PNG,480598LajoyaAlo-03_1728x.png,"{'x': 0.5, 'y': 0.5, 'source': 3}","{'topLeftAverage': 'e2deee', 'topRightAverage'...",638x709
3,66ff7538a4c24b619b59d961,66e00bac4c639c1118427a9c,11,1728017719959,1729068772687,8,False,False,[66ff6b514c76fd46af3cc5d6],1,...,{},{},store-item,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,66ff734efb783433b9728adc,66e00bac4c639c1118427a9c,11,1728017230419,1729007218492,6,False,False,[66ff6b514c76fd46af3cc5d6],1,...,{},{},store-item,701d9762-397f-4418-a588-38891ab7c28c,"1362x1512,100w,300w,500w,750w,1000w",PNG,El_Vergel_Gesha_Spirits_1512x.png,"{'x': 0.5, 'y': 0.5, 'source': 3}","{'topLeftAverage': 'f7bbbe', 'topRightAverage'...",1362x1512
5,66ff71c64c76fd46af3cc858,66e00bac4c639c1118427a9c,11,1728016838527,1729007176116,5,False,False,[66ff6b514c76fd46af3cc5d6],1,...,{},{},store-item,0b0475dc-2677-48d9-b36c-fbf1921ad3ca,"1362x1512,100w,300w,500w,750w,1000w",PNG,El_Vergel_Genius_Bourbon_dda3d30d-01f6-4cb3-a8...,"{'x': 0.5, 'y': 0.5, 'source': 3}","{'topLeftAverage': 'eabfc2', 'topRightAverage'...",1362x1512
6,66ff714960a18077259e217d,66e00bac4c639c1118427a9c,11,1728016713579,1729007053054,4,False,False,[66ff6b514c76fd46af3cc5d6],1,...,{},{},store-item,95204bdd-5c0a-4894-8cc5-a26af310cad0,"1359x1511,100w,300w,500w,750w,1000w",PNG,El_Fuerte_FRONT1_1512x.png,"{'x': 0.5, 'y': 0.5, 'source': 3}","{'topLeftAverage': 'fad2c2', 'topRightAverage'...",1359x1511
7,66ff70c260a18077259e2143,66e00bac4c639c1118427a9c,11,1728016578313,1729007048703,3,False,False,[66ff6b514c76fd46af3cc5d6],1,...,{},{},store-item,7e46f750-7847-4b8b-aade-b0ebc8bb8abe,"1360x1510,100w,300w,500w,750w,1000w",PNG,Alo_Washed_FRONT1_1024x1024@2x.png,"{'x': 0.5, 'y': 0.5, 'source': 3}","{'topLeftAverage': 'fbdbdd', 'topRightAverage'...",1360x1510
8,66ff63ddfb783433b972837d,66e00bac4c639c1118427a9c,11,1728013277482,1729007045095,0,False,False,[66ff6b514c76fd46af3cc5d6],1,...,{},{},store-item,d0be2c2e-ce6b-4db8-82e3-dc857541cff8,"1296x1440,100w,300w,500w,750w,1000w",PNG,Alo_COE_FRONT1_1296x_1.png,"{'x': 0.5, 'y': 0.5, 'source': 3}","{'topLeftAverage': 'e9e5f3', 'topRightAverage'...",1296x1440
9,66ff6e2160a18077259e202e,66e00bac4c639c1118427a9c,11,1728015905740,1730391400487,2,False,False,[66ff6b56fb783433b97287e1],1,...,{},{},store-item,700fe526-b2dc-4aba-be71-7ab57f9c31fb,"638x709,100w,300w,500w",PNG,480598LajoyaAlo-07_2808x.png,"{'x': 0.5, 'y': 0.5, 'source': 3}","{'topLeftAverage': 'fbf8c4', 'topRightAverage'...",638x709
